In [42]:
import numpy as np
import pandas as pd
# from utils import *

import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [43]:
metadata = pd.read_csv('AlphaGenome_metadata.csv')
metadata

,name,strand,Assay title,ontology_curie,biosample_name,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,output_type,gtex_tissue,histone_mark,transcription_factor
0,CL:0000084 ATAC-seq,.,ATAC-seq,CL:0000084,T-cell,primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN
1,CL:0000100 ATAC-seq,.,ATAC-seq,CL:0000100,motor neuron,in_vitro_differentiated_cells,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN
2,CL:0000236 ATAC-seq,.,ATAC-seq,CL:0000236,B cell,primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN
3,CL:0000623 ATAC-seq,.,ATAC-seq,CL:0000623,natural killer cell,primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN
4,CL:0000624 ATAC-seq,.,ATAC-seq,CL:0000624,"CD4-positive, alpha-beta T cell",primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5558,ENCSR182QNJ,-,PRO-cap,EFO:0001099,Caco-2,cell_line,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN
5559,ENCSR740IPL,-,PRO-cap,EFO:0002067,K562,cell_line,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN
5560,ENCSR797DEF,-,PRO-cap,EFO:0002819,Calu3,cell_line,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN
5561,ENCSR801ECP,-,PRO-cap,CL:0002618,endothelial cell of umbilical vein,primary_cell,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN


In [44]:
metadata['cell_type'] = metadata['biosample_name']
metadata['assay'] = pd.NA

mask = (metadata['Assay title'] == 'ATAC-seq')
metadata.loc[mask, 'assay'] = 'ATAC'
mask = (metadata['Assay title'] == 'DNase-seq')
metadata.loc[mask, 'assay'] = 'DNase'
mask = (metadata['Assay title'] == 'total RNA-seq')
metadata.loc[mask, 'assay'] = 'RNA-seq'
mask = (metadata['Assay title'] == 'hCAGE')
metadata.loc[mask, 'assay'] = 'CAGE'
mask = (metadata['Assay title'] == 'TF ChIP-seq')
metadata.loc[mask, 'assay'] = metadata.loc[mask, 'transcription_factor']
mask = (metadata['Assay title'] == 'Histone ChIP-seq')
metadata.loc[mask, 'assay'] = metadata.loc[mask, 'histone_mark']

metadata['index'] = metadata.index

In [45]:
metadata['genetically_modified'].value_counts()

genetically_modified
False    3150
True      734
Name: count, dtype: int64

In [46]:
metadata = metadata[(metadata['genetically_modified'] == False)| (pd.isna(metadata['genetically_modified']))]
metadata

,name,strand,Assay title,ontology_curie,biosample_name,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,output_type,gtex_tissue,histone_mark,transcription_factor,cell_type,assay,index
0,CL:0000084 ATAC-seq,.,ATAC-seq,CL:0000084,T-cell,primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN,T-cell,ATAC,0
1,CL:0000100 ATAC-seq,.,ATAC-seq,CL:0000100,motor neuron,in_vitro_differentiated_cells,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN,motor neuron,ATAC,1
2,CL:0000236 ATAC-seq,.,ATAC-seq,CL:0000236,B cell,primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN,B cell,ATAC,2
3,CL:0000623 ATAC-seq,.,ATAC-seq,CL:0000623,natural killer cell,primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN,natural killer cell,ATAC,3
4,CL:0000624 ATAC-seq,.,ATAC-seq,CL:0000624,"CD4-positive, alpha-beta T cell",primary_cell,adult,encode,paired,False,OutputType.ATAC,NaN,NaN,NaN,"CD4-positive, alpha-beta T cell",ATAC,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5558,ENCSR182QNJ,-,PRO-cap,EFO:0001099,Caco-2,cell_line,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN,Caco-2,<NA>,5558
5559,ENCSR740IPL,-,PRO-cap,EFO:0002067,K562,cell_line,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN,K562,<NA>,5559
5560,ENCSR797DEF,-,PRO-cap,EFO:0002819,Calu3,cell_line,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN,Calu3,<NA>,5560
5561,ENCSR801ECP,-,PRO-cap,CL:0002618,endothelial cell of umbilical vein,primary_cell,NaN,encode,NaN,False,OutputType.PROCAP,NaN,NaN,NaN,endothelial cell of umbilical vein,<NA>,5561


In [47]:
df_pivot = metadata.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)

df_pivot

assay,AFF1,AGO1,AGO2,ARID1B,ARID3A,ARNT,ASH1L,ASH2L,ATAC,ATF1,ATF2,ATF3,ATF4,ATF7,BACH1,BATF,BCL11A,BCL3,BCLAF1,BCOR,BHLHE40,BMI1,BRCA1,BRD4,BRD9,BRF2,C11orf30,CAGE,CBFA2T2,CBFA2T3,CBFB,CBX1,CBX3,CBX5,CC2D1A,CEBPB,CHAMP1,CHD1,CHD2,CHD4,CLOCK,COPS2,CREB1,CREB3L1,CREM,CTBP1,CTBP2,CTCF,CUX1,Cebpa,DACH1,DDX20,DEAF1,DNase,DPF2,E2F1,E2F4,E2F6,E2F8,E4F1,EBF1,EED,EGR1,EHMT2,ELF1,ELF4,ELK1,ELK4,EP300,EP400,ESRRA,ETS1,ETV4,ETV6,EZH2,EZH2phosphoT487,FOS,FOSL1,FOSL2,FOXA1,FOXA2,FOXK2,FOXM1,FUS,GABPA,GABPB1,GATA1,GATA2,GATA3,GATA4,GATAD2A,GATAD2B,GMEB1,GTF2F1,H2AFZ,H2AK5ac,H2BK120ac,H2BK12ac,H2BK15ac,H2BK20ac,H2BK5ac,H3F3A,H3K14ac,H3K18ac,H3K23ac,H3K27ac,H3K27me3,H3K36me3,H3K4ac,H3K4me1,H3K4me2,H3K4me3,H3K56ac,H3K79me1,H3K79me2,H3K9ac,H3K9me1,H3K9me2,H3K9me3,H4K20me1,H4K5ac,H4K8ac,H4K91ac,HCFC1,HDAC1,HDAC2,HDAC3,HDGF,HES1,HLTF,HMBOX1,HNF1A,HNF4A,HNF4G,HNRNPL,IKZF1,IKZF2,IRF2,IRF4,JUN,JUNB,JUND,KDM1A,KDM4B,KDM5A,KLF16,KLF5,L3MBTL2,LARP7,LEF1,MAFF,MAFK,MAX,MAZ,MBD2,MCM5,MEF2A,MEIS2,MGA,MIER1,MITF,MLLT1,MNT,MTA1,MTA2,MTA3,MXI1,MYBL2,MYC,MYNN,NANOG,NBN,NCOA1,NCOA3,NCOR1,NEUROD1,NFATC1,NFATC3,NFE2,NFE2L2,NFIB,NFIC,NFRKB,NFYA,NFYB,NKRF,NONO,NR0B1,NR2C1,NR2C2,NR2F1,NR2F2,NR2F6,NR3C1,NRF1,PAX5,PBX3,PCBP1,PHF20,PHF8,PKNOX1,PML,POLR2A,POLR2AphosphoS2,POLR2AphosphoS5,POLR2B,POLR2G,POLR3A,POU2F2,POU5F1,PRDM1,PRDM10,RAD21,RAD51,RB1,RBBP5,RBFOX2,RBM17,RBM22,RCOR1,RELA,RELB,REST,RFX1,RFX5,RNA-seq,RNF2,RUNX3,RXRA,SAFB,SAFB2,SETDB1,SIN3A,SIN3B,SIX5,SKI,SKIL,SMAD1,SMAD2,SMAD5,SMARCA4,SMARCA5,SMARCB1,SMARCC1,SMARCC2,SMARCE1,SMC3,SOX13,SOX6,SP1,SP4,SPI1,SRF,SRSF1,SRSF9,STAT3,STAT5A,SUPT20H,SUPT5H,SUZ12,SYNCRIP,TAF1,TAF7,TAF9B,TAL1,TARDBP,TBL1XR1,TBP,TBX21,TBX3,TCF12,TCF3,TCF7,TCF7L2,TEAD4,TFAP4,THAP1,TOE1,TRIM22,TRIM24,TRIM28,U2AF1,U2AF2,UBTF,USF1,USF2,XRCC4,XRCC5,YY1,ZBED1,ZBTB11,ZBTB2,ZBTB33,ZBTB40,ZBTB7A,ZC3H8,ZEB2,ZFP36,ZFP91,ZFX,ZHX2,ZKSCAN1,ZMIZ1,ZMYM3,ZNF184,ZNF207,ZNF217,ZNF24,ZNF263,ZNF274,ZNF282,ZNF316,ZNF318,ZNF384,ZNF407,ZNF444,ZNF574,ZNF579,ZNF592,ZNF639,ZNF687,ZSCAN29
cell_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22Rv1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[3950],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2095],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[319, 592]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[834],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [48]:
df_pivot.to_csv('AlphaGenome_pivot_table.csv')
df_pivot

assay,AFF1,AGO1,AGO2,ARID1B,ARID3A,ARNT,ASH1L,ASH2L,ATAC,ATF1,ATF2,ATF3,ATF4,ATF7,BACH1,BATF,BCL11A,BCL3,BCLAF1,BCOR,BHLHE40,BMI1,BRCA1,BRD4,BRD9,BRF2,C11orf30,CAGE,CBFA2T2,CBFA2T3,CBFB,CBX1,CBX3,CBX5,CC2D1A,CEBPB,CHAMP1,CHD1,CHD2,CHD4,CLOCK,COPS2,CREB1,CREB3L1,CREM,CTBP1,CTBP2,CTCF,CUX1,Cebpa,DACH1,DDX20,DEAF1,DNase,DPF2,E2F1,E2F4,E2F6,E2F8,E4F1,EBF1,EED,EGR1,EHMT2,ELF1,ELF4,ELK1,ELK4,EP300,EP400,ESRRA,ETS1,ETV4,ETV6,EZH2,EZH2phosphoT487,FOS,FOSL1,FOSL2,FOXA1,FOXA2,FOXK2,FOXM1,FUS,GABPA,GABPB1,GATA1,GATA2,GATA3,GATA4,GATAD2A,GATAD2B,GMEB1,GTF2F1,H2AFZ,H2AK5ac,H2BK120ac,H2BK12ac,H2BK15ac,H2BK20ac,H2BK5ac,H3F3A,H3K14ac,H3K18ac,H3K23ac,H3K27ac,H3K27me3,H3K36me3,H3K4ac,H3K4me1,H3K4me2,H3K4me3,H3K56ac,H3K79me1,H3K79me2,H3K9ac,H3K9me1,H3K9me2,H3K9me3,H4K20me1,H4K5ac,H4K8ac,H4K91ac,HCFC1,HDAC1,HDAC2,HDAC3,HDGF,HES1,HLTF,HMBOX1,HNF1A,HNF4A,HNF4G,HNRNPL,IKZF1,IKZF2,IRF2,IRF4,JUN,JUNB,JUND,KDM1A,KDM4B,KDM5A,KLF16,KLF5,L3MBTL2,LARP7,LEF1,MAFF,MAFK,MAX,MAZ,MBD2,MCM5,MEF2A,MEIS2,MGA,MIER1,MITF,MLLT1,MNT,MTA1,MTA2,MTA3,MXI1,MYBL2,MYC,MYNN,NANOG,NBN,NCOA1,NCOA3,NCOR1,NEUROD1,NFATC1,NFATC3,NFE2,NFE2L2,NFIB,NFIC,NFRKB,NFYA,NFYB,NKRF,NONO,NR0B1,NR2C1,NR2C2,NR2F1,NR2F2,NR2F6,NR3C1,NRF1,PAX5,PBX3,PCBP1,PHF20,PHF8,PKNOX1,PML,POLR2A,POLR2AphosphoS2,POLR2AphosphoS5,POLR2B,POLR2G,POLR3A,POU2F2,POU5F1,PRDM1,PRDM10,RAD21,RAD51,RB1,RBBP5,RBFOX2,RBM17,RBM22,RCOR1,RELA,RELB,REST,RFX1,RFX5,RNA-seq,RNF2,RUNX3,RXRA,SAFB,SAFB2,SETDB1,SIN3A,SIN3B,SIX5,SKI,SKIL,SMAD1,SMAD2,SMAD5,SMARCA4,SMARCA5,SMARCB1,SMARCC1,SMARCC2,SMARCE1,SMC3,SOX13,SOX6,SP1,SP4,SPI1,SRF,SRSF1,SRSF9,STAT3,STAT5A,SUPT20H,SUPT5H,SUZ12,SYNCRIP,TAF1,TAF7,TAF9B,TAL1,TARDBP,TBL1XR1,TBP,TBX21,TBX3,TCF12,TCF3,TCF7,TCF7L2,TEAD4,TFAP4,THAP1,TOE1,TRIM22,TRIM24,TRIM28,U2AF1,U2AF2,UBTF,USF1,USF2,XRCC4,XRCC5,YY1,ZBED1,ZBTB11,ZBTB2,ZBTB33,ZBTB40,ZBTB7A,ZC3H8,ZEB2,ZFP36,ZFP91,ZFX,ZHX2,ZKSCAN1,ZMIZ1,ZMYM3,ZNF184,ZNF207,ZNF217,ZNF24,ZNF263,ZNF274,ZNF282,ZNF316,ZNF318,ZNF384,ZNF407,ZNF444,ZNF574,ZNF579,ZNF592,ZNF639,ZNF687,ZSCAN29
cell_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22Rv1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[3950],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2095],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[319, 592]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[834],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [8]:
# df_pivot = pd.read_csv('AlphaGenome_pivot_table.csv', index_col=0)
# df_pivot

In [49]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549', 'GM12878', 'MCF-7', 'H1', 'HeLa-S3', 'IMR-90', 'B cell', 'GM23338', 'CD4-positive, alpha-beta T cell', 'HL-60', 'PC-3', 'Panc1']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']
df_pivot.loc[cell_types, assays]

assay,DNase,H3K4me3,H3K27ac,CTCF
cell_type,,,,
K562,[831],[2082],[2077],[3689]
HepG2,[822],[2028],[2023],[3065]
SK-N-SH,[866],[2201],[2196],[4194]
HCT116,[859],[2155],[2150],[4128]
A549,[820],[2016],[2013],[2870]
GM12878,[850],[2135],[2131],[3981]
MCF-7,[826],[2064],[2060],[3577]
H1,[863],[2183],[2178],[4161]
HeLa-S3,[855],[2144],[2139],[4081]


In [50]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF', 'RNA-seq', 'ATAC']
cell_types = ['K562', 'HepG2', 'HEK293', 'HEK293T', 'Jurkat', 'Jurkat, Clone E6-1', 'CD4-positive, alpha-beta T cell']
df_pivot.loc[cell_types, assays]

assay,DNase,H3K4me3,H3K27ac,CTCF,RNA-seq,ATAC
cell_type,,,,,,
K562,[831],[2082],[2077],[3689],"[1138, 1409, 4562, 4929]",[60]
HepG2,[822],[2028],[2023],[3065],"[1128, 1399, 4551, 4918]",[56]
HEK293,NaN,[2022],NaN,[2909],NaN,NaN
HEK293T,NaN,NaN,NaN,NaN,NaN,NaN
Jurkat,NaN,NaN,NaN,NaN,NaN,NaN
"Jurkat, Clone E6-1",[808],[2008],NaN,NaN,NaN,NaN
"CD4-positive, alpha-beta T cell",[731],[1835],[1831],[2815],"[1046, 1317, 4456, 4823]",[4]


In [51]:
# df_pivot.loc['Jurkat'] = df_pivot.loc['Jurkat']

In [18]:
Zhang_AlphaGenome_pred = load_h5('Zhang_AlphaGenome_pred.h5', key='DNase')
print(Zhang_AlphaGenome_pred.shape)

(17464, 2048, 305)


In [19]:
pred = Zhang_AlphaGenome_pred[:, 924:1124].mean(axis=1)
np.save('Zhang_AlphaGenome_pred_DNase.npy', pred)

# pred = Zhang_AlphaGenome_pred[:, 7:9].mean(axis=1)
# np.save('Zhang_AlphaGenome_pred_Histone.npy', pred)

# pred = Zhang_AlphaGenome_pred[:, 7:9].mean(axis=1)
# np.save('Zhang_AlphaGenome_pred_TF.npy', pred)

In [ ]:
# Zhang_AlphaGenome_pred = load_h5('Zhang_AlphaGenome_pred.h5')
# for key in Zhang_AlphaGenome_pred.keys():
#     print(key)

In [52]:
Zhang_AlphaGenome_pred_DNase = np.load('Zhang_AlphaGenome_pred_DNase.npy')
print(Zhang_AlphaGenome_pred_DNase.shape)

Zhang_AlphaGenome_pred_Histone = np.load('Zhang_AlphaGenome_pred_Histone.npy')
print(Zhang_AlphaGenome_pred_Histone.shape)

Zhang_AlphaGenome_pred_TF = np.load('Zhang_AlphaGenome_pred_TF.npy')
print(Zhang_AlphaGenome_pred_TF.shape)

(17400, 305)
(17400, 1116)
(17400, 1617)


In [53]:
MPRA_df = pd.read_csv('Zhang_MPRA_final_200bp.csv')
print(MPRA_df.shape)

(17400, 6)


In [59]:
type(df_pivot.loc['K562', 'DNase'])

list

In [54]:
def safe_eval(x):
    if pd.isna(x):   # 如果是 NaN，直接返回 NaN
        return np.nan
    return ast.literal_eval(x)

In [64]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']
cell_types = ['K562', 'HepG2', 'HEK293', 'HEK293T', 'Jurkat', 'Jurkat, Clone E6-1', 'CD4-positive, alpha-beta T cell']
VEF_df = pd.DataFrame()

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        
        indice = df_pivot.loc[cell_type, assay]

        # 解析 index
        if isinstance(indice, str):
            try:
                indice = ast.literal_eval(indice)
            except Exception:
                indice = None
        elif not isinstance(indice, list):
            indice = None

        if indice is None:
            VEF_df[f'{cell_type}_{assay}'] = np.nan
            continue

        if assay == 'DNase':
            print()
            indice = [i-713 for i in indice]
            pred = Zhang_AlphaGenome_pred_DNase[:, indice].mean(1)
        elif assay in ['H3K4me3', 'H3K27ac']:
            indice = [i-1685 for i in indice]
            pred = Zhang_AlphaGenome_pred_Histone[:, indice].mean(1)
        # elif assay == 'CTCF':
        #     MPRA_df[f'{cell_type}_{assay}'] = np.nan
        #     continue
        #     print(pred)
        #     VEF_df[f'{cell_type}_{assay}'] = pred
        # else:
        #     VEF_df[f'{cell_type}_{assay}'] = np.nan
            
        VEF_df[f'{cell_type}_{assay}'] = pred

VEF_df

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HEK293_DNase,HEK293_H3K4me3,HEK293_H3K27ac,HEK293_CTCF,HEK293T_DNase,HEK293T_H3K4me3,HEK293T_H3K27ac,HEK293T_CTCF,Jurkat_DNase,Jurkat_H3K4me3,Jurkat_H3K27ac,Jurkat_CTCF,"Jurkat, Clone E6-1_DNase","Jurkat, Clone E6-1_H3K4me3","Jurkat, Clone E6-1_H3K27ac","Jurkat, Clone E6-1_CTCF","CD4-positive, alpha-beta T cell_DNase","CD4-positive, alpha-beta T cell_H3K4me3","CD4-positive, alpha-beta T cell_H3K27ac","CD4-positive, alpha-beta T cell_CTCF"
0,0.110,30.812,104.750,104.750,2.463,69.250,854.000,854.000,NaN,53.875,NaN,53.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108,20.188,NaN,NaN,0.027,29.188,45.500,45.500
1,0.009,2.500,3.180,3.180,0.021,3.766,4.000,4.000,NaN,2.051,NaN,2.051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.020,2.711,NaN,NaN,0.006,3.352,5.906,5.906
2,0.164,29.938,82.000,82.000,1.005,60.500,612.000,612.000,NaN,23.062,NaN,23.062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.091,18.750,NaN,NaN,0.055,18.562,25.062,25.062
3,0.026,7.391,14.344,14.344,0.765,40.750,1440.000,1440.000,NaN,4.672,NaN,4.672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030,5.500,NaN,NaN,0.007,7.344,14.375,14.375
4,0.130,26.688,57.250,57.250,0.516,44.250,136.500,136.500,NaN,59.500,NaN,59.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.085,22.125,NaN,NaN,0.024,19.812,35.250,35.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,0.054,12.812,25.188,25.188,0.995,25.812,160.500,160.500,NaN,20.688,NaN,20.688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071,11.188,NaN,NaN,0.032,17.000,28.000,28.000
17396,0.010,2.578,3.141,3.141,0.012,3.078,2.250,2.250,NaN,2.031,NaN,2.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016,2.117,NaN,NaN,0.007,2.523,4.188,4.188
17397,0.009,3.609,6.031,6.031,0.038,5.500,9.188,9.188,NaN,4.266,NaN,4.266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016,2.922,NaN,NaN,0.004,3.914,6.766,6.766
17398,0.084,15.219,48.125,48.125,1.875,22.688,526.000,526.000,NaN,12.281,NaN,12.281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050,8.062,NaN,NaN,0.017,13.344,26.562,26.562


In [ ]:
MPRA_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_seq_feature_alphagenome_raw.csv', index=False)